In [ ]:
!pip install encord encord-agents

# SETUP

In [ ]:
import os

from google.colab import userdata

## VPC/US Clients

In [ ]:
os.environ["ENCORD_SSH_KEY"] = userdata.get("encord_vpc_key")
DOMAIN = "https://api.annotation.encord.ai"

## EMEA Clients

In [ ]:
os.environ["ENCORD_SSH_KEY"] = userdata.get("encord_key")
DOMAIN = "https://api.encord.com"

In [ ]:
from encord import EncordUserClient

os.environ["ENCORD_DOMAIN"] = DOMAIN

# In the SECRETS tab on the left (key icon), add your Encord SSH key
# You can get your Encord Key here: https://docs.encord.com/platform-documentation/Annotate/annotate-api-keys
encord_client = EncordUserClient.create_with_ssh_private_key(ssh_private_key=userdata.get("encord_key"), domain=DOMAIN)

In [ ]:
# Change this to be your Project Hash
PROJECT_HASH = "00000000-0000-0000-0000-000000000000"
STAGE_NAME = "Custom Router"

project = encord_client.get_project(PROJECT_HASH)

# Filter Labels based on Object Attributes

In [ ]:
ONTOLOGY_OBJECT_NAME = "Bird"
ONTOLOGY_OBJECT_SUBFIELD_NAME = "Type"
ATTRIBUTE_TO_FILTER_ON = "Bluejay"

In [ ]:
# define ontology (& attributes you'd like)
ontology = project.ontology_structure
ontology_object = ontology.get_child_by_title(ONTOLOGY_OBJECT_NAME)
question = ontology_object.get_child_by_title(ONTOLOGY_OBJECT_SUBFIELD_NAME)

field_filter = ontology_object.get_child_by_title(ATTRIBUTE_TO_FILTER_ON)

# Agent Code

In [ ]:
from encord_agents.core.utils import get_user_client
from encord_agents.tasks import Depends, Runner

runner = Runner(project_hash=PROJECT_HASH)

In [ ]:
from encord.objects import LabelRowV2
from encord.workflow.stages.agent import AgentStage, AgentTask


# main agent
@runner.stage(STAGE_NAME, overwrite=True)
def move_to_next_stage(lr: LabelRowV2):
    for object in lr.get_object_instances(ontology_object):
        try:
            if object.get_answer(question).feature_node_hash == field_filter.feature_node_hash:
                return "Pass"
        except Exception:
            continue
    return "Second Review"

In [ ]:
# Run the Agent
# After 50 label updates, tasks are moved in the workflow.
# Set to refresh every 3600 seconds to continuously check for new tasks
runner(refresh_every=3600, task_batch_size=50, max_tasks_per_stage=None)